In [1]:
import boto3
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import joblib
import sagemaker
from sagemaker.sklearn import SKLearn
from io import BytesIO
from tqdm import tqdm 
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [2]:
# Configuración inicial
s3_bucket = "challenge-cvs-test-target"
output_s3 = "challenge-csvstest"

# Crear cliente S3
s3 = boto3.client('s3')
print(s3)

In [3]:
# Obtener la lista de archivos Parquet en el bucket
response = s3.list_objects_v2(Bucket=s3_bucket, Prefix="")
parquet_files = [obj['Key'] for obj in response.get('Contents', []) if obj['Key'].endswith('.parquet')]

In [4]:
# Diccionario para agrupar por esquema de columnas
schemas = {}

# Leer y agrupar los archivos Parquet
for file_key in parquet_files:
    # Descargar el archivo Parquet desde S3
    response = s3.get_object(Bucket=s3_bucket, Key=file_key)
    parquet_data = BytesIO(response['Body'].read())

    # Leer el archivo Parquet en un DataFrame
    df = pd.read_parquet(parquet_data)

    # Crear una clave única basada en las columnas
    column_schema = tuple(df.columns)

    # Agrupar los DataFrames con el mismo esquema de columnas
    if column_schema not in schemas:
        schemas[column_schema] = []
    schemas[column_schema].append(df)

# Concatenar los DataFrames dentro de cada grupo
dataframes = [pd.concat(dfs, ignore_index=True) for dfs in schemas.values()]

# Resultados: dos DataFrames con diferentes esquemas
if len(dataframes) == 2:
    vacantes, candidatos = dataframes
else:
    print("Se encontraron más o menos de dos esquemas.")
    for i, df in enumerate(dataframes):
        print(f"DataFrame {i+1}:\n", df.head())

In [5]:
print(vacantes.columns)
print(vacantes.shape)

Index(['ID_Vacante', 'Titulo_Puesto', 'Descripcion', 'Requisitos'], dtype='object')
(30000, 4)


In [6]:
print(candidatos.columns)
print(candidatos.shape)

Index(['ID_Candidato', 'Nombre', 'Email', 'Telefono', 'Direccion',
       'ID_Experiencia', 'Empresa', 'Titulo_Puesto', 'Duracion',
       'Habilidades_Utilizadas'],
      dtype='object')
(50000, 10)


In [7]:
vacantes_reducido = vacantes.sample(20, random_state=42)
candidatos_reducido = candidatos.sample(10000, random_state=42)

# Emparejar las vacantes seleccionadas con todos los candidatos
combinacion = vacantes_reducido.merge(candidatos_reducido, how='cross')

In [8]:
combinacion.shape

(200000, 14)

In [9]:
# Función para calcular etiquetas (simulación de puntuaciones históricas)
def calcular_puntaje(row):
    req_habilidades = set(row['Requisitos'].split(", "))
    cand_habilidades = set(row['Habilidades_Utilizadas'].split(", "))
    interseccion = req_habilidades.intersection(cand_habilidades)
    return len(interseccion) / len(req_habilidades) * 100 if req_habilidades else 0

In [10]:
# Etiqueta: Ranking de compatibilidad
combinacion['Ranking_Compatibilidad'] = combinacion.apply(calcular_puntaje, axis=1)

# Seleccionar columnas relevantes
X = combinacion[['Requisitos', 'Habilidades_Utilizadas', 'Titulo_Puesto_x', 'Duracion']]
y = combinacion['Ranking_Compatibilidad']

# Vectorizar texto con TF-IDF
tfidf = TfidfVectorizer()
X_tfidf_requisitos = tfidf.fit_transform(X['Requisitos']).toarray()
X_tfidf_habilidades = tfidf.transform(X['Habilidades_Utilizadas']).toarray()

In [11]:
print(X_tfidf_requisitos.shape)
print(X_tfidf_habilidades.shape)

(200000, 9)
(200000, 9)


In [12]:
# Combinar representaciones TF-IDF con otras características numéricas
X_combined = pd.concat([
    pd.DataFrame(X_tfidf_requisitos, columns=tfidf.get_feature_names_out()),
    pd.DataFrame(X_tfidf_habilidades, columns=[f"H_{col}" for col in tfidf.get_feature_names_out()]),
    X[['Duracion']].reset_index(drop=True)
], axis=1)

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_combined, y, test_size=0.2, random_state=42)

# Entrenar un modelo Random Forest
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestRegressor(random_state=42)

In [13]:
# Predicción y evaluación
y_pred = model.predict(X_test)
rmse = mean_squared_error(y_test, y_pred, squared=False)
print(f"RMSE del modelo: {rmse}")

RMSE del modelo: 8.816197511263557


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [14]:
# Predicción en nuevos datos
vacantes_reducido2 = vacantes.sample(20, random_state=42)
candidatos_reducido2 = candidatos.sample(10000, random_state=42)

# Emparejar las vacantes seleccionadas con todos los candidatos
nueva_combinacion = vacantes_reducido2.merge(candidatos_reducido2, how='cross')
nueva_combinacion['Ranking_Predicho'] = model.predict(X_combined)
print(nueva_combinacion[['ID_Vacante', 'ID_Candidato', 'Ranking_Predicho']])

        ID_Vacante  ID_Candidato  Ranking_Predicho
0            23805         68351         35.739484
1            23805         75937         24.937500
2            23805          1617         50.806250
3            23805            99         25.000000
4            23805         15936         32.840142
...            ...           ...               ...
199995        6837         28323         66.666667
199996        6837          1236          0.000000
199997        6837         50468          0.000000
199998        6837         22375         66.666667
199999        6837         47111          0.000000

[200000 rows x 3 columns]


In [15]:
nueva_combinacion.shape

(200000, 15)

In [16]:
def calcular_coincidencia(row):
    habilidades_candidato = set(row['Habilidades_Utilizadas'].lower().split(','))
    requisitos_puesto = set(row['Requisitos'].lower().split(','))
    
    # Calcular la intersección de habilidades
    coincidencias = habilidades_candidato.intersection(requisitos_puesto)
    
    # Calcular el porcentaje de coincidencia
    if len(requisitos_puesto) > 0:
        return (len(coincidencias) / len(requisitos_puesto)) * 100
    else:
        return 0

In [17]:
# Aplicar la función a cada fila del DataFrame para crear la columna 'Coincidencia_Habilidades (%)'
nueva_combinacion['Coincidencia_Habilidades (%)'] = nueva_combinacion.apply(calcular_coincidencia, axis=1)

In [18]:
nueva_combinacion.columns

Index(['ID_Vacante', 'Titulo_Puesto_x', 'Descripcion', 'Requisitos',
       'ID_Candidato', 'Nombre', 'Email', 'Telefono', 'Direccion',
       'ID_Experiencia', 'Empresa', 'Titulo_Puesto_y', 'Duracion',
       'Habilidades_Utilizadas', 'Ranking_Predicho',
       'Coincidencia_Habilidades (%)'],
      dtype='object')

In [19]:
resultados = nueva_combinacion.sample(50000, random_state=42)

In [27]:
# Supongamos que 'nueva_combinacion' es tu DataFrame
resultados.to_excel('Resultados.xlsx')


In [28]:
buffer = BytesIO()
buffer.seek(0)
s3_file_name = 'Resultados.xlsx'
# Subir el archivo al bucket de S3
s3.upload_fileobj(buffer, output_s3, s3_file_name)

print(f'Archivo {s3_file_name} subido correctamente al bucket {output_s3}')

Archivo Resultados.xlsx subido correctamente al bucket challenge-csvstest
